# Выбор региона для разработки новых нефтяных месторождений

Для нефтедобывающей компании необходимо решить, где бурить новую скважину.  
Собраны характеристики пробы нефти для скважин: качество нефти и объём её запасов по трем регионам. Характеристики для каждой скважины в регионе уже известны.   

Необходимо построить модель машинного обучения для предсказания объёма запасов в новых скважинах.  
Определить регионы с максимальной суммарной прибылью отобранных скважин.  
Построенную модель использоваться для определения региона, где добыча принесёт наибольшую прибыль.  

Результаты исследования проанализировать техникой Bootstrap на возможную прибыль и риски

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span><ul class="toc-item"><li><span><a href="#Разделим-данные-регионов-на-обучающую-и-валидационную-выборки-в-соотношении-75:25." data-toc-modified-id="Разделим-данные-регионов-на-обучающую-и-валидационную-выборки-в-соотношении-75:25.-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Разделим данные регионов на обучающую и валидационную выборки в соотношении 75:25.</a></span></li><li><span><a href="#Обучим-модель-и-сделаем-предсказания-на-валидационной-выборке." data-toc-modified-id="Обучим-модель-и-сделаем-предсказания-на-валидационной-выборке.-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обучим модель и сделаем предсказания на валидационной выборке.</a></span></li><li><span><a href="#Сохраним-предсказания-и-правильные-ответы-на-валидационной-выборке." data-toc-modified-id="Сохраним-предсказания-и-правильные-ответы-на-валидационной-выборке.-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Сохраним предсказания и правильные ответы на валидационной выборке.</a></span></li><li><span><a href="#Выведем-на-экран-средний-запас-предсказанного-сырья-и-RMSE-модели." data-toc-modified-id="Выведем-на-экран-средний-запас-предсказанного-сырья-и-RMSE-модели.-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Выведем на экран средний запас предсказанного сырья и RMSE модели.</a></span></li><li><span><a href="#Анализ-результатов." data-toc-modified-id="Анализ-результатов.-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Анализ результатов.</a></span></li></ul></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span><ul class="toc-item"><li><span><a href="#Все-ключевые-значения-для-расчётов-сохраним-в-отдельных-переменных." data-toc-modified-id="Все-ключевые-значения-для-расчётов-сохраним-в-отдельных-переменных.-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Все ключевые значения для расчётов сохраним в отдельных переменных.</a></span></li><li><span><a href="#Рассчитаем-достаточный-объём-сырья-для-безубыточной-разработки-новой-скважины." data-toc-modified-id="Рассчитаем-достаточный-объём-сырья-для-безубыточной-разработки-новой-скважины.-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.</a></span></li><li><span><a href="#Выводы-по-этапу-подготовки-расчёта-прибыли." data-toc-modified-id="Выводы-по-этапу-подготовки-расчёта-прибыли.-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Выводы по этапу подготовки расчёта прибыли.</a></span></li></ul></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span></li><li><span><a href="#Подсчет-риска-и-прибыли" data-toc-modified-id="Подсчет-риска-и-прибыли-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Подсчет риска и прибыли</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

## Загрузка и подготовка данных

Пути к файлам датасетов для каждого региона:
- `/datasets/geo_data_0.csv`
- `/datasets/geo_data_1.csv`
- `/datasets/geo_data_2.csv`

Импортируем все необходимые библиотеки для проекта

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy import stats as st
import numpy as np

Загрузим и посмотрим на содержание всех датасетов для каждого региона

In [2]:
region_0 = pd.read_csv('/datasets/geo_data_0.csv')
region_1 = pd.read_csv('/datasets/geo_data_1.csv')
region_2 = pd.read_csv('/datasets/geo_data_2.csv')

display(region_0.head())
print(region_0.shape)

display(region_1.head())
print(region_1.shape)

display(region_2.head())
print(region_2.shape)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


(100000, 5)


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


(100000, 5)


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


(100000, 5)


- `id` — уникальный идентификатор скважины;
- `f0`, `f1`, `f2` — три признака точек;
- `product` — объём запасов в скважине (тыс. баррелей).

Количество запсией в каждом датасете `100_000`

Колонка с `id` не понадобится и обучать модель с ней не стоит. Отбросим колонку с `id` для всех датасетов.

In [3]:
region_0.drop('id', axis=1, inplace=True)
region_1.drop('id', axis=1, inplace=True)
region_2.drop('id', axis=1, inplace=True)

display(region_0.head())
display(region_1.head())
display(region_2.head())

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746


## Обучение и проверка модели

### Разделим данные регионов на обучающую и валидационную выборки в соотношении 75:25.

`Регион 0`

In [4]:
reg0_features = region_0.drop(['product'], axis=1)
reg0_target = region_0['product']

reg0_features_train, reg0_features_valid, reg0_target_train, reg0_target_valid = train_test_split(
    reg0_features, reg0_target, test_size=0.25, random_state=12345)

`Регион 1`

In [5]:
reg1_features = region_1.drop(['product'], axis=1)
reg1_target = region_1['product']

reg1_features_train, reg1_features_valid, reg1_target_train, reg1_target_valid = train_test_split(
    reg1_features, reg1_target, test_size=0.25, random_state=12345)

`Регион 2`

In [6]:
reg2_features = region_2.drop(['product'], axis=1)
reg2_target = region_2['product']

reg2_features_train, reg2_features_valid, reg2_target_train, reg2_target_valid = train_test_split(
    reg2_features, reg2_target, test_size=0.25, random_state=12345)

### Обучим модель и сделаем предсказания на валидационной выборке.

Для каждого региона отдельно создадим и обучим модель

In [7]:
reg0_model = LinearRegression()
reg0_model.fit(reg0_features_train, reg0_target_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [8]:
reg1_model = LinearRegression()
reg1_model.fit(reg1_features_train, reg1_target_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [9]:
reg2_model = LinearRegression()
reg2_model.fit(reg2_features_train, reg2_target_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### Сохраним предсказания и правильные ответы на валидационной выборке.

Полученные предсказания для каждого региона сохраним по отдельности, предварительно сконвертировав в `pd.Series`

In [10]:
reg0_predictions_valid = pd.Series(reg0_model.predict(reg0_features_valid))

In [11]:
reg1_predictions_valid = pd.Series(reg1_model.predict(reg1_features_valid))

In [12]:
reg2_predictions_valid = pd.Series(reg2_model.predict(reg2_features_valid))

### Выведем на экран средний запас предсказанного сырья и RMSE модели.

Создадим функцию `region_mean_rmse` для подсчета, предсказания среднего запаса сырья и метрики RMSE.

In [13]:
def region_mean_rmse(region, reg_target_valid, reg_predictions_valid):
    mean_valid = reg_target_valid.mean()
    mean_predic = reg_predictions_valid.mean()
    rmse = mean_squared_error(reg_target_valid, reg_predictions_valid)**0.5

    print(f"Для региона: {region} \n"
          f"Фактический средний запас сырья: {mean_valid}\n"
          f"Предсказанный средний запас сырья на валидационной выборке: {mean_predic}\n"
          f"RMSE модели линейной регрессии на валидационной выборке: {rmse}\n")

In [14]:
region_mean_rmse("0", reg0_target_valid, reg0_predictions_valid)

Для региона: 0 
Фактический средний запас сырья: 92.07859674082927
Предсказанный средний запас сырья на валидационной выборке: 92.59256778438038
RMSE модели линейной регрессии на валидационной выборке: 37.5794217150813



In [15]:
region_mean_rmse("1", reg1_target_valid, reg1_predictions_valid)

Для региона: 1 
Фактический средний запас сырья: 68.72313602435997
Предсказанный средний запас сырья на валидационной выборке: 68.728546895446
RMSE модели линейной регрессии на валидационной выборке: 0.893099286775616



In [16]:
region_mean_rmse("2", reg2_target_valid, reg2_predictions_valid)

Для региона: 2 
Фактический средний запас сырья: 94.88423280885438
Предсказанный средний запас сырья на валидационной выборке: 94.96504596800489
RMSE модели линейной регрессии на валидационной выборке: 40.02970873393434



### Анализ результатов.

- Данные для 3-х регионов были разделены на обучающие и тестовые выборки в соотношении 75:25.  
- Проведено обучение модели линейной регрессии и получены предсказанные значения запасов сырья.  
- Для `Региона 0` и `Региона 2` среднее значение запасов сырья выше, чем в `Регионе 1`.    
- Но при этом для `Региона 1` метрика RMSE показала гораздо лучшее значение, всего `0.8930`, чем в двух других регионах. Но и средний запас сырья модель предсказала самый низкий.

## Подготовка к расчёту прибыли

### Все ключевые значения для расчётов сохраним в отдельных переменных.

`BUDGET` - Бюджет на разработку скважин в регионе 10 млрд рублей.  
`PROFIT` - Доход с каждой тысячи баррелей составляет 450 тыс. рублей.  
`EXPLORED_OILWELLS` - Исследованные нефтяные скважины при разведке региона.  
`TOP_OILWELLS` - ТОП лучших скважин для разработки.

In [17]:
BUDGET = 10_000_000_000
PROFIT = 450_000
EXPLORED_OILWELLS = 500
TOP_OILWELLS = 200

### Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.

In [18]:
sufficient_amount = BUDGET / PROFIT / TOP_OILWELLS
print(f"Достаточный объём сырья для безубыточной разработки новой скважины: "
      f"{round(sufficient_amount, 2)} тыс. баррелей")

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11 тыс. баррелей


Сравним полученный объём сырья со средним запасом в каждом регионе.

In [19]:
print(f"Средний запас сырья:\n"
      f"- для Региона 0: {round(reg0_target.mean(), 2)} тыс. баррелей\n"
      f"- для Региона 1: {round(reg1_target.mean(), 2)} тыс. баррелей\n"
      f"- для Региона 2: {round(reg2_target.mean(), 2)} тыс. баррелей\n"
     )

Средний запас сырья:
- для Региона 0: 92.5 тыс. баррелей
- для Региона 1: 68.83 тыс. баррелей
- для Региона 2: 95.0 тыс. баррелей



### Выводы по этапу подготовки расчёта прибыли.

С учетом достаточного объема сырья в `111 тысяч баррелей` для безубыточной разработки скважины, в среднем по всем трём регионам, объем скважин **убыточен**. Наименьший средний запас сырья в `Регионе 1`, а наибольший в `Регионе 2`.

## Расчёт прибыли и рисков 

Напишем функцию `revenue` для расчёта прибыли по выбранным скважинам и предсказаниям модели.
Выберем скважины с максимальными значениями предсказаний, просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.

In [20]:
def revenue(target, predictions, count):
    predicted = predictions.sort_values(ascending=False).head(count)
    selected = target.reset_index(drop=True)[predicted.index]
    return (PROFIT * selected.sum() - BUDGET)

Рассчитаем прибыль для полученного объёма сырья.

In [21]:
print(f"Прибыль для полученного объёма сырья с ТОП-{TOP_OILWELLS} скважин:\n"
      f"- для Региона 0: {revenue(reg0_target_valid, reg0_predictions_valid, TOP_OILWELLS):,} руб.\n"
      f"- для Региона 1: {revenue(reg1_target_valid, reg1_predictions_valid, TOP_OILWELLS):,} руб.\n"
      f"- для Региона 2: {revenue(reg2_target_valid, reg2_predictions_valid, TOP_OILWELLS):,} руб.\n"
     )

Прибыль для полученного объёма сырья с ТОП-200 скважин:
- для Региона 0: 3,320,826,043.1398525 руб.
- для Региона 1: 2,415,086,696.681511 руб.
- для Региона 2: 2,710,349,963.5998325 руб.



Из двухсот скважин с максимальным значением предсказаний, более прибыльным можно отметить `Регион 0`, а менее прибыльным `Регион 1`.  
Но так ли это, проведем подсчет риска и прибыли на следующем шаге.

## Подсчет риска и прибыли

Посчитаем риски и прибыль для каждого региона применив технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

Напишем функцию `bootstrap_revenue` для подсчета прибыли в связке с функцией `bootstrap`

In [22]:
def bootstrap_revenue(target, predictions, count):
    predicted = predictions.sort_values(ascending=False)
    selected = target[predicted.index][:count]
    return (PROFIT * selected.sum() - BUDGET)

В функции `bootstrap` найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. Результаты выведем на экран.

In [23]:
def bootstrap(target, predictions, explored_oilwells, top_oilwells):
    state = np.random.RandomState(12345)
    values = []

    for i in range(1000):
        target_subsample = target.reset_index(drop=True).sample(n=explored_oilwells, replace=True, random_state=state)
        probs_subsample = predictions[target_subsample.index]
        values.append(bootstrap_revenue(target_subsample, probs_subsample, top_oilwells))

    values = pd.Series(values)
    negative_revenue = (values < 0).sum() / len(values) * 100
    lower = round(values.quantile(0.025), 2)
    upper = round(values.quantile(0.975), 2)

    print(f"Средняя прибыль: {values.mean():,} рублей\n"
          f"95%-ый доверительный интервал: {lower:,} ... {upper:,} рублей\n"
          f"Процент риска убытков в регионе: {negative_revenue}%")

Риски и прибыль для `Региона 0`

In [24]:
bootstrap(reg0_target_valid, reg0_predictions_valid, EXPLORED_OILWELLS, TOP_OILWELLS)

Средняя прибыль: 425,938,526.9105923 рублей
95%-ый доверительный интервал: -102,090,094.84 ... 947,976,353.36 рублей
Процент риска убытков в регионе: 6.0%


Риски и прибыль для `Региона 1`

In [25]:
bootstrap(reg1_target_valid, reg1_predictions_valid, EXPLORED_OILWELLS, TOP_OILWELLS)

Средняя прибыль: 515,222,773.44328994 рублей
95%-ый доверительный интервал: 68,873,225.37 ... 931,547,591.26 рублей
Процент риска убытков в регионе: 1.0%


Риски и прибыль для `Региона 2`

In [26]:
bootstrap(reg2_target_valid, reg2_predictions_valid, EXPLORED_OILWELLS, TOP_OILWELLS)

Средняя прибыль: 435,008,362.78275555 рублей
95%-ый доверительный интервал: -128,880,547.33 ... 969,706,954.18 рублей
Процент риска убытков в регионе: 6.4%


## Выводы

Проведя исследования данных трех регионов, обучив модели линейной регресии и проверив возможные не только прибыль но и риски, остановим выбор на `Регионе 1`, имеено он подходит для разработки месторождения.<br>

В этом регионе получилась наибольшая средняя выручка с 1 скважины `515,222,773` рублей и доля скважин, которые могут принести убыток всего `1 %`. <br>
95% доверительный интервал говорит о том, что существует высокая вероятность того, что реальная прибыль будет в пределах диапазона `68,873,225` - `931,547,591` рублей.